In [13]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver

In [14]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [15]:
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = llm.invoke(messages)

    # response store state
    return {'messages': [response]}

In [20]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer= checkpointer)

In [21]:
# chatbot

In [6]:
initial_state = {
    'messages': [HumanMessage(content='What is the capital of india')]
}

chatbot.invoke(initial_state)['messages'][-1].content

'The capital of India is **New Delhi**.'

In [22]:
thread_id= '1' #it is the id that is assigned to a particular conversation
while True:
    user_message= input("User: ")
    print("User:", user_message)

    if user_message.strip().lower() in ['exit', 'quit', 'bye']:
        break
    
    config= {
        'configurable':{ 'thread_id': thread_id}
    }

    initial_state = {
    'messages': [HumanMessage(content= user_message)]
    }
    response= chatbot.invoke(initial_state, config= config)

    print("AI: ", response['messages'][-1].content)

#in this memory is not getting retain because we are using for loop and every time initial state is getting reset
#as full workflow is exexuted from start to end in every loop  

User: hi my name is daniyal
AI:  Hi Daniyal, it's nice to meet you! How can I help you today?
User: what is my name
AI:  Your name is Daniyal. You just told me! 😊
User: thanks
AI:  You're welcome, Daniyal! Is there anything else I can help you with?
User: no
AI:  Okay, have a great day, Daniyal!
User: thanks
AI:  You're very welcome! Take care.
User: bye
